# Vasant Dave
# O41154429
# April 3 ,2024

Structure of the notebook :                                                    

1) Installation and Summary breakdown for the whole code and its workings

2) Code with self explainatory comments

3) Some flaws or challenges and reference

Goal of the notebook: Exploration how NEAT (NeuroEvolution of Augmenting Topologies) can evolve FNN(Feed forward neural networks) to be implemented in game development, an autonomous AI that can beat any score a human has achieved).

In [ ]:
#Installation needed if working with them for the first time
!pip install neat-python
!pip install pygame

Please see that the directory to load the image and sound assets has been defined correctly , also that that Config.Feedforward.txt file directory and presence.

In [ ]:
# Constants
# Define various game parameters such as screen dimensions, gravity, pipe speed, etc.
# SCREEN_WIDTH and SCREEN_HEIGHT represent the dimensions of the game window.
# GROUND_HEIGHT represents the height of the ground in the game.
# GRAVITY represents the acceleration due to gravity acting on the bird.
# PIPE_SPEED represents the speed at which pipes move horizontally.
# FLAP_POWER represents the upward force applied to the bird when it flaps.
# GAP_SIZE represents the vertical gap between pipes.
# DRAW_LINES is a boolean flag indicating whether to draw collision detection lines.
# Other constants represent image filenames, sound files, and game settings.
# FPS (Frames Per Second) controls the speed of the game.
# GEN represents the current generation in the genetic algorithm.
# The constants are used throughout the code to maintain consistency and adjust game parameters easily.

# Create the screen
# Initialize the Pygame window with the specified dimensions.
# Set the caption of the window to 'Flappy Bird'.

# Load images and sounds
# Load images for pipes, background, birds, and base.
# Load sound effects for flap, collision, and point scoring.
# Set the volume for each sound effect.

# Global variables
# Initialize a global variable 'gen' to keep track of the current generation.

# Class definitions
# Define classes for the Bird, Pipe, and Base objects.
# These classes encapsulate the behavior and attributes of game elements.

# Bird class
# Represents the flappy bird character in the game.
# It has attributes like position, velocity, images, etc.
# Methods include jump(), move(), draw(), and get_mask().

# Pipe class
# Represents the pipes that the bird needs to navigate through.
# It has attributes like position, height, images, etc.
# Methods include set_height(), move(), draw(), collide(), and pass_through().

# Base class
# Represents the moving floor of the game.
# It has attributes like position, velocity, image, etc.
# Methods include move() and draw().

# Function to draw game window
# Draws various game elements like background, pipes, birds, base, and score.
# Also displays the current generation and number of alive birds.

# Function to evaluate genomes
# Evaluates the fitness of each genome (bird) in the population.
# Uses NEAT (NeuroEvolution of Augmenting Topologies) to evolve the population.
# Manages the game loop, handling user inputs and updating the game state.

# Function to run NEAT algorithm
# Initializes NEAT algorithm with specified configuration file.
# Runs the algorithm to evolve the population and find the best genome (bird).

# Entry point of the script
# Calls the run() function with the configuration file path.











#Import necessary libraries 
#Pygame the main one with the built in methods from creating screen to event handling
#Random for Randomising the pipe spawns.
#OS for file read and directory fetch
#Time uses to record the frames per second , also various other operations including checking the current generation and 

import pygame
import random
import os
import time
import neat



# Initialize Pygame
pygame.font.init()
pygame.mixer.init()

# Constants 
SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 800
GROUND_HEIGHT = 25
FLOOR = SCREEN_HEIGHT - GROUND_HEIGHT
GRAVITY = 0.25
PIPE_SPEED = 3
FLAP_POWER = -9
GAP_SIZE = 200
DRAW_LINES = True

COLLISION_RADIUS = 0.25
PIPE_SPAWN_RATE = 150
GAME_SPEED_COUNTER_POS = (10, 100)
FPS = 90



# Create the screen
WIN = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption('Flappy Bird')

# Load images
pipe_img = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "pipe.png")).convert_alpha())
bg_img = pygame.transform.scale(pygame.image.load(os.path.join("imgs", "bg.png")).convert_alpha(), (SCREEN_WIDTH, SCREEN_HEIGHT))
bird_images = [pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "bird" + str(x) + ".png"))) for x in range(1, 4)]
base_img = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "base.png")).convert_alpha())
STAT_FONT = pygame.font.SysFont("comicsans", 50)

flap_sound = pygame.mixer.Sound(os.path.join("sounds", "flap_sound.wav"))
collision_sound = pygame.mixer.Sound(os.path.join("sounds", "collision_sound.wav"))
point_sound = pygame.mixer.Sound(os.path.join("sounds", "point_sound.wav"))

#Set flap volume
flap_sound.set_volume(0.1)
collision_sound.set_volume(0.1)
point_sound.set_volume(0.3)

# Global variables
gen = 0

class Bird:
    """
    Class representing the flappy bird
    """
    MAX_ROTATION = 25
    ROT_VEL = 20
    ANIMATION_TIME = 5
    IMGS = bird_images

    def __init__(self, x, y):
        """
        Initialize the bird object
        :param x: starting x pos (int)
        :param y: starting y pos (int)
        """
        self.x = x
        self.y = y
        self.vel = 0
        self.tick_count = 0
        self.height = self.y
        self.img_count = 0
        self.img = self.IMGS[0]
        self.tilt = 0

    def jump(self):
        """
        Make the bird jump
        """
        self.vel = FLAP_POWER
        self.tick_count = 0
        self.height = self.y
        flap_sound.play()

    def move(self):
        """
        Make the bird move
        """
        self.tick_count += 1
        displacement = self.vel * self.tick_count + 0.5 * 3 * self.tick_count ** 2

        if displacement >= 16:
            displacement = 16
        if displacement < 0:
            displacement -= 2

        self.y = self.y + displacement

        if self.y < 0:
            self.y = 0

        if displacement < 0 or self.y < self.height + 50:
            if self.tilt < self.MAX_ROTATION:
                self.tilt = self.MAX_ROTATION
        else:
            if self.tilt > -90:
                self.tilt -= self.ROT_VEL

    def draw(self, win):
        """
        Draw the bird
        """
        self.img_count += 1

        if self.img_count <= self.ANIMATION_TIME:
            self.img = self.IMGS[0]
        elif self.img_count <= self.ANIMATION_TIME * 2:
            self.img = self.IMGS[1]
        elif self.img_count <= self.ANIMATION_TIME * 3:
            self.img = self.IMGS[2]
        elif self.img_count <= self.ANIMATION_TIME * 4:
            self.img = self.IMGS[1]
        elif self.img_count == self.ANIMATION_TIME * 4 + 1:
            self.img = self.IMGS[0]
            self.img_count = 0

        if self.tilt <= -80:
            self.img = self.IMGS[1]
            self.img_count = self.ANIMATION_TIME * 2

        rotated_img = pygame.transform.rotate(self.img, self.tilt)
        new_rect = rotated_img.get_rect(center=self.img.get_rect(topleft=(self.x, self.y)).center)
        win.blit(rotated_img, new_rect.topleft)

    def get_mask(self):
        """
        Get the mask for the current image of the bird
        """
        return pygame.mask.from_surface(self.img)

class Pipe:
    """
    Represents a pipe object
    """
    GAP = GAP_SIZE
    VEL = PIPE_SPEED

    def __init__(self, x):
        """
        Initialize pipe object
        """
        self.x = x
        self.height = 0
        self.top = 0
        self.bottom = 0
        self.PIPE_TOP = pygame.transform.flip(pipe_img, False, True)
        self.PIPE_BOTTOM = pipe_img
        self.passed = False
        self.set_height()

    def set_height(self):
        """
        Set the height of the pipe
        """
        self.height = random.randrange(50, SCREEN_HEIGHT - GAP_SIZE - GROUND_HEIGHT)
        self.top = self.height - self.PIPE_TOP.get_height()
        self.bottom = self.height + self.GAP

    def move(self):
        """
        Move pipe based on velocity
        """
        self.x -= self.VEL

    def draw(self, win):
        """
        Draw the pipe
        """
        win.blit(self.PIPE_TOP, (self.x, self.top))
        win.blit(self.PIPE_BOTTOM, (self.x, self.bottom))

    def collide(self, bird):
        """
        Returns if a point is colliding with the pipe
        """
        bird_mask = bird.get_mask()
        top_mask = pygame.mask.from_surface(self.PIPE_TOP)
        bottom_mask = pygame.mask.from_surface(self.PIPE_BOTTOM)
        top_offset = (self.x - bird.x, self.top - round(bird.y))
        bottom_offset = (self.x - bird.x, self.bottom - round(bird.y))

        b_point = bird_mask.overlap(bottom_mask, bottom_offset)
        t_point = bird_mask.overlap(top_mask, top_offset)

        if b_point or t_point:
            collision_sound.play()
            return True

        return False

    def pass_through(self, bird):
        """
        Check if the bird passes through the pipe
        """
        if not self.passed and self.x < bird.x:
            self.passed = True
            point_sound.play()
            return True
        return False

class Base:
    """
    Represents the moving floor of the game
    """
    VEL = PIPE_SPEED
    WIDTH = base_img.get_width()
    IMG = base_img

    def __init__(self, y):
        """
        Initialize the object
        """
        self.y = y
        self.x1 = 0
        self.x2 = self.WIDTH

    def move(self):
        """
        Move floor so it looks like it's scrolling
        """
        self.x1 -= self.VEL
        self.x2 -= self.VEL
        if self.x1 + self.WIDTH < 0:
            self.x1 = self.x2 + self.WIDTH

        if self.x2 + self.WIDTH < 0:
            self.x2 = self.x1 + self.WIDTH

    def draw(self, win):
        """
        Draw the floor
        """
        win.blit(self.IMG, (self.x1, self.y))
        win.blit(self.IMG, (self.x2, self.y))

def draw_window(win, birds, pipes, base, score, alive_birds, generation):
    """
    Draw the game window
    """
    win.blit(bg_img, (0, 0))

    for pipe in pipes:
        pipe.draw(win)

    base.draw(win)
    for bird in birds:
        bird.draw(win)
        if DRAW_LINES:
            # Draw lines indicating collision detection
            bird_center = (bird.x + bird.img.get_width() / 2, bird.y + bird.img.get_height() / 2)
            top_pipe_edge = (pipes[0].x, pipes[0].height)
            bottom_pipe_edge = (pipes[0].x, pipes[0].bottom)
            pygame.draw.line(win, (255, 0, 0), bird_center, top_pipe_edge, 5)
            pygame.draw.line(win, (255, 0, 0), bird_center, bottom_pipe_edge, 5)

    score_label = STAT_FONT.render("Score: " + str(score), 1, (255, 255, 255))
    win.blit(score_label, (SCREEN_WIDTH - score_label.get_width() - 15, 10))

    alive_label = STAT_FONT.render("Alive: " + str(alive_birds), 1, (255, 255, 255))
    win.blit(alive_label, (10, 10))

    gen_label = STAT_FONT.render("Generation: " + str(generation), 1, (255, 255, 255))
    win.blit(gen_label, (10, 50))

    pygame.display.update()


def eval_genomes(genomes, config):
    """
    Evaluate the fitness of each genome
    """
    global WIN
    win = WIN

    nets = []
    birds = []
    ge = []
    generation = 0  # Initialize generation counter
    for genome_id, genome in genomes:
        genome.fitness = 0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        nets.append(net)
        bird = Bird(230, 350)
        birds.append(bird)
        ge.append(genome)

    base = Base(FLOOR)
    pipes = [Pipe(700)]
    score = 0

    clock = pygame.time.Clock()   #Clock method to clock frames per second

    run = True
    while run and len(birds) > 0:
        clock.tick(FPS)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                quit()
                break

        pipe_ind = 0
        if len(birds) > 0:
            if len(pipes) > 1 and birds[0].x > pipes[0].x + pipes[0].PIPE_TOP.get_width():
                pipe_ind = 1

        for x, bird in enumerate(birds):
            ge[x].fitness += 0.1
            bird.move()

            output = nets[birds.index(bird)].activate(
                (bird.y, abs(bird.y - pipes[pipe_ind].height), abs(bird.y - pipes[pipe_ind].bottom)))

            if output[0] > 0.5:
                bird.jump()

        base.move()

        rem = []
        add_pipe = False
        for pipe in pipes:
            pipe.move()

            for bird in birds:
                if pipe.collide(bird):
                    ge[birds.index(bird)].fitness -= 1
                    nets.pop(birds.index(bird))
                    ge.pop(birds.index(bird))
                    birds.pop(birds.index(bird))

            if pipe.pass_through(bird):
                add_pipe = True

            if pipe.x + pipe.PIPE_TOP.get_width() < 0:
                rem.append(pipe)

        if add_pipe:
            score += 1
            for genome in ge:
                genome.fitness += 5
            pipes.append(Pipe(SCREEN_WIDTH))

        for r in rem:
            pipes.remove(r)

        for bird in birds:
            if bird.y + bird.img.get_height() - 10 >= FLOOR or bird.y < -50:
                nets.pop(birds.index(bird))
                ge.pop(birds.index(bird))
                birds.pop(birds.index(bird))

        draw_window(WIN, birds, pipes, base, score, len(birds), generation)  # Pass generation number
        generation = gen 

        
def run(config_file):
    """
    Run the NEAT algorithm
    """
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    p = neat.Population(config)

    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)

    winner = p.run(eval_genomes, 50)

    print('\nBest genome:\n{!s}'.format(winner))

if __name__ == '__main__':
    config_path = 'config-feedforward.txt'
    run(config_path)

Some flaws with the sounds [no matter what I try , the sound degradation takes place due to the Many birds used to train thee neural network]


There is some issues with the sound overlapping when there are many birds so sometimes the point score sound does not sound.
Additionally, the game speed is also a factor for the sound overlapping or parsing.
I can change that to just the sound of a single bird to make the point score sound everytime,but then more flap sounds will be removed.
but to make it proper, I made the volume equation to lessen volume as per the amount of birds.
so to summarize , there are still problems with the *sound* for the point sometimes.

# Reference https://www.youtube.com/@TechWithTim/videos  [Tech with Tim] 